In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [ ]:
from proj1_helpers import *
from helpers2 import *
from preprocessing import *

In [ ]:
#data_folder = Path("../data/")
DATA_TRAIN_PATH = "../data/train.csv"
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Data analysis

In [ ]:
print(y)
print(np.shape(y))
print(np.shape(tX))
print(tX.dtype)
print(y.dtype)

In [ ]:
feature_names = ['DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 
                 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt', 
                 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt', 
                 'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 
                 'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi',
                 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt']

for i, name in enumerate(feature_names):
    print(i, feature_names[i])


The training data set is composed of : 
* a y vector of length 250'000 and type float
* a tX float matrix of 250'000 rows and 30 columns

It means that our data set is composed of 250'000 different obsevations of 30 different features. In the rest of the notebook, we name the features by their index nummer. So, it means from the feature 0 from the feature  29.

In [ ]:
plt.hist(y)
plt.title('Histogram of y')
plt.show()

There is more y = -1 than y = 1 in the data, so there is more y = 'b' than y = 's'. So, we have to pay attention to normalize the data in order to compare them in the next plots.

In [ ]:
plt.figure(figsize=(20, 10))
ind = np.arange(30)
plt.boxplot(tX[:,], labels = ind)

plt.show()

There are many outliers depending on the feature. There are also feature that has a long interquantile range. Maybe we have to treat these feature in order to be more efficient in our futur predictions. Let's do more plots to be have a better idea :

In [ ]:
ind_1 = np.where(y == 1)
ind_2 = np.where(y == -1)
tX_1 = tX[ind_1[0],:]
tX_2 = tX[ind_2[0],:]

fig, axs = plt.subplots(5, 6, figsize=(25,20))

n = 0
for i in range(5) :
    for j in range(6) :
        axs[i,j].hist(tX_2[:,n], alpha=0.4, density=True, label=['y = -1'])
        axs[i,j].hist(tX_1[:,n], alpha=0.4, density=True, label=['y = 1'])
        axs[i,j].legend()
        axs[i,j].set_title(n)
        n = n + 1
plt.show()

By plotting the histograms of the features with a color for each y, we can see that there are useless features as they have almost the same distribution for y=1 than for y = -1. We can cut feature 15, 18, 20. 


There are also features that are very inequally distributed with value that are about -1000 and values around 0 ; it can be problematic for the prediction with such a large gap between values of a single distribution. Moreover, there is not a big difference in the distribution of y=1 and y=-1. Maybe it can be useful to put off these big negative values of these features. The features in question are : 0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28.


In [ ]:
fig, axs = plt.subplots(5, 6, figsize=(20,20))

n = 0
for i in range(5) :
    for j in range(6) :
        axs[i,j].scatter(tX[:,n], y)
        axs[i,j].set_title(n)
        n = n + 1
plt.show()

These plots confirm the obsevations that we made in the previous plot ; no difference of the distribution of y for features 15, 18, 20 and very large gap in the distributions of features : 0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28.

## Data filtering

### Outliers

In [ ]:
def nb_outliers(tX, outlier) : 
    sum = 0
    nb_outliers = []
    for col in range(tX.shape[1]) :
        sum = np.where(tX[:,col] == outlier)[0].shape
        nb_outliers.append(sum)   
    print(nb_outliers)
    print(np.where(tX==outlier)[0].shape)

out = -999

print('number of -999 in the entire matrix :')
nb_outliers(tX, out)

ind_1 = np.where(y == 1)
ind_2 = np.where(y == -1)
tX_1 = tX[ind_1[0],:]
tX_2 = tX[ind_2[0],:]

print('number of -999 in the rows where y = 1 :')
nb_outliers(tX_1, out)
print('number of -999 in the rows where y = -1 :')
nb_outliers(tX_2, out)

We have a problem with features  0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28. They are inequally distributed; they have a lot of -999 values and the rest is values around 0. So, here we can see how much of these -999 there are. We can see that the -999 appear only in the features that we identified with the histograms. It seems that there is a correlation between features as many features have the same number of -999. We can also see that there is more -999 in the obsevations where y=-1, so we have to take this into account when we filter the data. As there are many -999, we can't delete the rows where there is -999 because we will loose to much information.

Explication sets

In [ ]:
def histograms(y, tX):
    ind_1 = np.where(y == 1)
    ind_2 = np.where(y == -1)
    tX_1 = tX[ind_1[0],:]
    tX_2 = tX[ind_2[0],:]

    fig, axs = plt.subplots(5, 6, figsize=(25,20))

    n = 0
    for i in range(5) :
        for j in range(6) :
            axs[i,j].hist(tX_2[:,n], alpha=0.4, density=True, label=['y = -1'])
            axs[i,j].hist(tX_1[:,n], alpha=0.4, density=True, label=['y = 1'])
            axs[i,j].legend()
            axs[i,j].set_title(n)
            n = n + 1
            if n>=tX.shape[1]: break
        if n>=tX.shape[1]: break
    plt.show()

In [ ]:
set1_x, set1_y, set1_ids, set2_x, set2_y, set2_ids, set3_x, set3_y, set3_ids = separate_sets(tX, y, ids)

print('Set 1')
set1_x = outliers(set1_x, -999)
_ = outliers(set1_x, -999)

print('\nSet 2')
set2_x = outliers(set2_x, -999)
_ = outliers(set2_x, -999)

print('\nSet 3')
set3_x = outliers(set3_x, -999)
_ = outliers(set3_x, -999)

In [ ]:
print('Set 1')
print(np.all(set1_x==set1_x[0], axis = 0))
histograms(set1_y, set1_x)

print('\nSet 2')
print(np.all(set2_x==set2_x[0], axis = 0))
histograms(set2_y, set2_x)

print('\nSet 3')
print(np.all(set3_x==set3_x[0], axis = 0))
histograms(set3_y, set3_x)

explanation things to delete 

TEST FOR y=[0,1]

In [ ]:
print(y)
y[y==-1]=0
print(y)

In order to filter the data, we want to get rif of these -999, but we can't just delete the rows. So, we have the idea to replace the -999 by the mean of the rest of values of the feature. As there is a significant difference of amount of -999 in between y=1 and y=-1 in certain features, we calculate the mean for the rows where y = 1 and y = -1 separatly.

Then, we can also standardize the data. It can be a good idea because the features are not all in the same range of values and it can create disproportionality between the importance of the features.

As we see above in the histograms, some features seem to be useless as they have a similar distribution between the y = 1 and y = -1. So, it is useful to have function that cut or keep some parameters.

In [ ]:
to_log = [1, 2, 5, 9, 10, 13, 16, 19, 21, 23, 26, 29]

## Fonctions 

In [ ]:
from implementations import *
from cross_validation import *

### Least squares

The best results for least squares regression is with the tX matrix which is standardize, filter with filtering_with_mean_bis function and whose features 15, 18, 20 are cut :

In [ ]:
tX_cut = cut(tX, [15,18,20])
x_essai = std(filtering_with_mean_bis(tX_cut, y))
degrees = np.arange(1,11)
# Cross-validation on the degrees
degree_opt, _ = best_degree_selection(y, x_essai, degrees, k_fold=10, lambdas=0, fonction=0)
print("Cross validation finished: optimal degree {d}".format(d=degree_opt))
# Best degree model
tX_poly = build_poly(x_essai, degree_opt)
w_ls, loss_ls = least_squares(y, tX_poly)
print("Least square loss rmse {loss}".format(loss=np.sqrt(loss_ls)))
degree_ls = degree_opt

The least squares best model on the entire is degree 1 ; its accuracy is : 0.606. Let's try the same model but applied to the 3 different sets according to the PRI_jet_num parameter thanks to the separate_sets function. We sould normally improve our results : 

In [ ]:
tX_cut = cut(tX, [15,18,20])

# Separation into set according to the PRI_jet_num parameter, as it is the 22th column of the entire matrix
# if we cut 3 paramaters before the 22th, PRI_jet_num is the 22-3th now
set1_x, set1_y, set1_ids, set2_x, set2_y, set2_ids, set3_x, set3_y, set3_ids = separate_sets(tX_cut, y, ids, 22-3)

def best_filtering_ls(set_x, set_y) :
        set_x = outliers(set_x, -999)
        set_x = std(filtering_with_mean_bis(set_x, set_y))
        return set_x

print('Set 1')
set1_x_ls = best_filtering_ls(set1_x, set1_y)

print('\nSet 2')
set2_x_ls = best_filtering_ls(set2_x, set2_y)

print('\nSet 3')
set3_x_ls = best_filtering_ls(set3_x, set3_y)

Let's do least squares regression for each set : 

In [ ]:
degrees = np.arange(1,11)
# Cross-validation on the degrees
degree_opt, _ = best_degree_selection(set1_y, set1_x_ls, degrees, k_fold=10, lambdas=0, fonction=0)
print("Cross validation finished: optimal degree {d}".format(d=degree_opt))
# Best degree model 
tX_poly = build_poly(set1_x_ls, degree_opt)
w_set1_ls, loss_ls = least_squares(set1_y, tX_poly)
print("Least square loss rmse {loss}".format(loss=np.sqrt(loss_ls)))
degree_set1_ls = degree_opt

In [ ]:
degrees = np.arange(1,11)
# Cross-validation on the degrees
degree_opt, _ = best_degree_selection(set2_y, set2_x_ls, degrees, k_fold=10, lambdas=0, fonction=0)
print("Cross validation finished: optimal degree {d}".format(d=degree_opt))
# Best degree model
tX_poly = build_poly(set2_x_ls, degree_opt)
w_set2_ls, loss_ls = least_squares(set2_y, tX_poly)
print("Least square loss rmse {loss}".format(loss=np.sqrt(loss_ls)))
degree_set2_ls = degree_opt

In [ ]:
degrees = np.arange(1,11)
# Cross-validation on the degrees
degree_opt, _ = best_degree_selection(set3_y, set3_x_ls, degrees, k_fold=10, lambdas=0, fonction=0)
print("Cross validation finished: optimal degree {d}".format(d=degree_opt))
# Best degree model
tX_poly = build_poly(set3_x_ls, degree_opt)
w_set3_ls, loss_ls = least_squares(set3_y, tX_poly)
print("Least square loss rmse {loss}".format(loss=np.sqrt(loss_ls)))
degree_set3_ls = degree_opt

The least square best model (standardize + filter_with_mean_bis + cut 15,18,20) with separate_sets is :
* set 1 : degree 1
* set 2 : degree 4
* set 3 : degree 5

We have a better accuracy : 0.683

### Ridge regression 

The best results for Ridge regression is with tX that is only filtered by filtering_with_mean_bis function : 

In [ ]:
degrees = np.arange(1,8)
x_essai = filtering_with_mean_bis(tX, y)
# Cross-validation on the degrees and the lambdas
degree_opt, lambda_opt = best_degree_selection(y, x_essai, degrees, k_fold=4, lambdas=np.logspace(-4, 0, 30), fonction=1)
print("Cross validation finished: optimal lambda {l} and degree {d}".format(l=lambda_opt, d=degree_opt))
# Best degree and lambda model
x_essai = build_poly(x_essai, degree_opt)
w_rr, loss_rr = ridge_regression(y, x_essai, lambda_opt)
print("Ridge regression loss {loss}".format(loss=loss_rr))
degree_rr = degree_opt

Let's look if we can improve the results with the separation of tX into sets according to the feature 22 (PRI_jet_num) : 

In [ ]:
set1_x, set1_y, set1_ids, set2_x, set2_y, set2_ids, set3_x, set3_y, set3_ids = separate_sets(tX, y, ids)

def best_filtering_rr(set_x, set_y) :
        set_x = outliers(set_x, -999)
        set_x = filtering_with_mean_bis(set_x, set_y)
        return set_x
    

print('Set 1')
set1_x_rr = best_filtering_rr(set1_x, set1_y)

print('\nSet 2')
set2_x_rr = best_filtering_rr(set2_x, set2_y)

print('\nSet 3')
set3_x_rr = best_filtering_rr(set3_x, set3_y)

Ridge regression according to the different sets : 

In [ ]:
degrees = np.arange(1,8)
x_essai = set1_x_rr
y = set1_y
# Cross-validation on the degrees and the lambdas
degree_opt, lambda_opt = best_degree_selection(y, x_essai, degrees, k_fold=4, lambdas=np.logspace(-4, 0, 30), fonction=1)
print("Cross validation finished: optimal lambda {l} and degree {d}".format(l=lambda_opt, d=degree_opt))
# Best degree and lambda model
x_essai = build_poly(x_essai, degree_opt)
w_rr_set1, loss_rr = ridge_regression(y, x_essai, lambda_opt)
print("Ridge regression loss {loss}".format(loss=loss_rr))
lambda_rr_set1 = lambda_opt
degree_rr_set1 = degree_opt

In [ ]:
degrees = np.arange(1,8)
x_essai = set2_x_rr
y = set2_y
# Cross-validation on the degrees and the lambdas
degree_opt, lambda_opt = best_degree_selection(y, x_essai, degrees, k_fold=4, lambdas=np.logspace(-4, 0, 30), fonction=1)
print("Cross validation finished: optimal lambda {l} and degree {d}".format(l=lambda_opt, d=degree_opt))
# Best degree and lambda model
x_essai = build_poly(x_essai, degree_opt)
w_rr_set2, loss_rr = ridge_regression(y, x_essai, lambda_opt)
print("Ridge regression loss {loss}".format(loss=loss_rr))
lambda_rr_set2 = lambda_opt
degree_rr_set2 = degree_opt

In [ ]:
degrees = np.arange(1,8)
x_essai = set3_x_rr
y = set3_y
# Cross-validation on the degrees and the lambdas
degree_opt, lambda_opt = best_degree_selection(y, x_essai, degrees, k_fold=4, lambdas=np.logspace(-4, 0, 30), fonction=1)
print("Cross validation finished: optimal lambda {l} and degree {d}".format(l=lambda_opt, d=degree_opt))
# Best degree and lambda model
x_essai = build_poly(x_essai, degree_opt)
w_rr_set3, loss_rr = ridge_regression(y, x_essai, lambda_opt)
print("Ridge regression loss {loss}".format(loss=loss_rr))
lambda_rr_set3 = lambda_opt
degree_rr_set3 = degree_opt

The least square best model (filter_with_mean_bis) with separate_sets is :
* set 1 : degree 1 and lambda = 0.0001
* set 2 : degree 4 and lambda = 0.0002592943797404667
* set 3 : degree 4 and lambda = 0.0002592943797404667

We have better results with the matrix that is separated in sets, so let's do a submission with this method. We have 0.801 of accuracy.

### Gradient descent

We process the data to get better results 

In [ ]:
k_fold = 4
max_iters = 500
gammas = np.arange(0, 0.5, 0.01)
tX_log = log_distribution(tX, to_log)

set1_x, set1_y, set1_ids, set2_x, set2_y, set2_ids, set3_x, set3_y, set3_ids = separate_sets(tX_log, y, ids)

set1_x = outliers(set1_x, -999)
set1_x = filtering_with_mean_bis(set1_x, set1_y)
#set1_x = filtering_with_mean(set1_x)
#set1_x = normalize(set1_x_gd)
set1_x = std(set1_x)

set2_x = outliers(set2_x, -999)
set2_x = filtering_with_mean_bis(set2_x, set2_y)
#set2_x = filtering_with_mean(set2_x)
#set2_x= normalize(set2_x)
set2_x = std(set2_x)

set3_x = outliers(set3_x, -999)
set3_x = filtering_with_mean_bis(set3_x, set3_y)
#set3_x = filtering_with_mean(set3_x)
#set2_x = normalize(set2_x)
set3_x = std(set3_x)
print('')
print("Preprocessing for gradient descent done!")

In the next cells, we perform a 4-fold cross validation for the gamma parameter for gradient descent method for each set. Then we we perform a stochastic gradient descent with the optimal gamma found. For the cross validation we use 50 iterations and for the final descent 500 iterations as we want a more precise final result. 

In [ ]:
initial_w = np.zeros(set1_x.shape[1])
gamma_opt1 = cross_validation(set1_y, set1_x, k_fold, gammas, fonction=2)
w_gd1, loss_gd1 = least_squares_GD(set1_y, set1_x, gamma_opt1, max_iters=max_iters)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt1))
print("Gradient descent regression loss {loss}".format(loss=loss_gd1))

In [ ]:
initial_w = np.zeros(set2_x.shape[1])
gamma_opt2 = cross_validation(set2_y, set2_x, k_fold, gammas, fonction=2)
w_gd2, loss_gd2 = least_squares_GD(set2_y, set2_x, gamma_opt2, max_iters=max_iters)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt2))
print("Gradient descent regression loss {loss}".format(loss=loss_gd2))

In [ ]:
initial_w = np.zeros(set3_x.shape[1])
gamma_opt3 = cross_validation(set3_y, set3_x, k_fold, gammas, fonction=2)
w_gd3, loss_gd3 = least_squares_GD(set3_y, set3_x, gamma_opt3, max_iters=max_iters)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt3))
print("Gradient descent regression loss {loss}".format(loss=loss_gd3))

### Stochastic gradient descent

In [104]:
k_fold = 4
max_iters = 500
gammas = np.arange(0, 0.5, 0.01)
tX_log = log_distribution(tX, to_log)
set1_x, set1_y, set1_ids, set2_x, set2_y, set2_ids, set3_x, set3_y, set3_ids = separate_sets(tX_log, y, ids)

set1_x = outliers(set1_x, -999)
set1_x = filtering_with_mean_bis(set1_x, set1_y)
#set1_x = filtering_with_mean(set1_x)
#set1_x = normalize(set1_x_gd)
set1_x = std(set1_x)

set2_x = outliers(set2_x, -999)
set2_x = filtering_with_mean_bis(set2_x, set2_y)
#set2_x = filtering_with_mean(set2_x)
#set2_x= normalize(set2_x)
set2_x = std(set2_x)

set3_x = outliers(set3_x, -999)
set3_x = filtering_with_mean_bis(set3_x, set3_y)
#set3_x = filtering_with_mean(set3_x)
#set2_x = normalize(set2_x)
set3_x = std(set3_x)
print('')
print("Preprocessing for stochastic gradient descent done!")

outliers ratio for each feature [0.2614574679971575, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
outliers ratio for each feature [0.09751882802022077, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
outliers ratio for each feature [0.06105344416415092, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Preprocessing for stochastic gradient descent done!


In the next cells, we perform a 4-fold cross validation for the gamma parameter for stochastic gradient descent method for each set. Then we we perform a stochastic gradient descent with the optimal gamma found. For the cross validation we use 50 iterations and for the final descent 500 iterations as we want a more precise final result. 

In [105]:
gamma_opt_sgd1 = cross_validation(set1_y, set1_x, k_fold, gammas, fonction=3)
w_sgd1, loss_sgd1 = least_squares_SGD(set1_y, set1_x, gamma_opt_sgd1, max_iters=500)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt_sgd1))
print("Stochastic gradient descent regression loss {loss}".format(loss=loss_sgd1))

SGD(0/49): loss=0.5
SGD(1/49): loss=0.5
SGD(2/49): loss=0.5
SGD(3/49): loss=0.5
SGD(4/49): loss=0.5
SGD(5/49): loss=0.5
SGD(6/49): loss=0.5
SGD(7/49): loss=0.5
SGD(8/49): loss=0.5
SGD(9/49): loss=0.5
SGD(10/49): loss=0.5
SGD(11/49): loss=0.5
SGD(12/49): loss=0.5
SGD(13/49): loss=0.5
SGD(14/49): loss=0.5
SGD(15/49): loss=0.5
SGD(16/49): loss=0.5
SGD(17/49): loss=0.5
SGD(18/49): loss=0.5
SGD(19/49): loss=0.5
SGD(20/49): loss=0.5
SGD(21/49): loss=0.5
SGD(22/49): loss=0.5
SGD(23/49): loss=0.5
SGD(24/49): loss=0.5
SGD(25/49): loss=0.5
SGD(26/49): loss=0.5
SGD(27/49): loss=0.5
SGD(28/49): loss=0.5
SGD(29/49): loss=0.5
SGD(30/49): loss=0.5
SGD(31/49): loss=0.5
SGD(32/49): loss=0.5
SGD(33/49): loss=0.5
SGD(34/49): loss=0.5
SGD(35/49): loss=0.5
SGD(36/49): loss=0.5
SGD(37/49): loss=0.5
SGD(38/49): loss=0.5
SGD(39/49): loss=0.5
SGD(40/49): loss=0.5
SGD(41/49): loss=0.5
SGD(42/49): loss=0.5
SGD(43/49): loss=0.5
SGD(44/49): loss=0.5
SGD(45/49): loss=0.5
SGD(46/49): loss=0.5
SGD(47/49): loss=0.5
SG

SGD(14/49): loss=0.5291240299989789
SGD(15/49): loss=0.49088487897180083
SGD(16/49): loss=0.46931943159915845
SGD(17/49): loss=0.4640410464201806
SGD(18/49): loss=0.4658127252423739
SGD(19/49): loss=0.4679665609841776
SGD(20/49): loss=0.46915736186936113
SGD(21/49): loss=0.5015153055078989
SGD(22/49): loss=0.49465421991951475
SGD(23/49): loss=0.48811142433675186
SGD(24/49): loss=0.4983912595025903
SGD(25/49): loss=0.49187573913759314
SGD(26/49): loss=0.4856675080115036
SGD(27/49): loss=0.49851488333797833
SGD(28/49): loss=0.5326348043568336
SGD(29/49): loss=0.5254330912687287
SGD(30/49): loss=0.5185527788468773
SGD(31/49): loss=0.5119858282295294
SGD(32/49): loss=0.5139280039982933
SGD(33/49): loss=0.517480924354712
SGD(34/49): loss=0.5157609676281986
SGD(35/49): loss=0.49590583731298693
SGD(36/49): loss=0.490242279439982
SGD(37/49): loss=0.4983936359025991
SGD(38/49): loss=0.4978418768450882
SGD(39/49): loss=0.4927684310472138
SGD(40/49): loss=0.49605062661995286
SGD(41/49): loss=0.50

SGD(1/49): loss=0.5183614882284064
SGD(2/49): loss=0.604308393789273
SGD(3/49): loss=0.5975853717325916
SGD(4/49): loss=0.5871068062697826
SGD(5/49): loss=0.6043894631170141
SGD(6/49): loss=0.6028830143093313
SGD(7/49): loss=0.5852930234281533
SGD(8/49): loss=0.5868069806391786
SGD(9/49): loss=0.5688054969599081
SGD(10/49): loss=0.5986093370642934
SGD(11/49): loss=0.6525213455312378
SGD(12/49): loss=0.6346761727921937
SGD(13/49): loss=0.6183508760570535
SGD(14/49): loss=0.5886507436438658
SGD(15/49): loss=0.6054193559442265
SGD(16/49): loss=0.6138187822252301
SGD(17/49): loss=0.6124126203051328
SGD(18/49): loss=0.6570491701753576
SGD(19/49): loss=0.6982901799055736
SGD(20/49): loss=0.5957195942595748
SGD(21/49): loss=0.6069989479477627
SGD(22/49): loss=0.5940931868268294
SGD(23/49): loss=0.7673526215917137
SGD(24/49): loss=0.740270900178604
SGD(25/49): loss=0.6550297404133736
SGD(26/49): loss=0.6335574875266938
SGD(27/49): loss=0.613640432348511
SGD(28/49): loss=0.5997674376791157
SGD(

SGD(45/49): loss=0.7445499267126436
SGD(46/49): loss=0.7484858333302012
SGD(47/49): loss=0.7538975314378988
SGD(48/49): loss=0.848430648473734
SGD(49/49): loss=0.9543277247251354
SGD(0/49): loss=0.5
SGD(1/49): loss=0.9424860594517448
SGD(2/49): loss=1.0066364147787332
SGD(3/49): loss=0.9242083917979695
SGD(4/49): loss=0.727715477328072
SGD(5/49): loss=0.6482228947330586
SGD(6/49): loss=0.5075945246559872
SGD(7/49): loss=0.4930490213908547
SGD(8/49): loss=0.48366446049693107
SGD(9/49): loss=0.5020793285758898
SGD(10/49): loss=0.5523266874049907
SGD(11/49): loss=0.5414769146059478
SGD(12/49): loss=0.5337730618908855
SGD(13/49): loss=0.5165989997496024
SGD(14/49): loss=0.5138175022939421
SGD(15/49): loss=0.5624086883522362
SGD(16/49): loss=0.649629865977146
SGD(17/49): loss=0.7342080963285026
SGD(18/49): loss=0.7540147701460229
SGD(19/49): loss=0.7701083311340213
SGD(20/49): loss=0.7907707858757292
SGD(21/49): loss=0.811699542805235
SGD(22/49): loss=0.6184373397361683
SGD(23/49): loss=0.6

SGD(31/49): loss=1.0674601431837565
SGD(32/49): loss=1.0216904242865847
SGD(33/49): loss=1.0298898875000573
SGD(34/49): loss=0.6480542935916845
SGD(35/49): loss=0.8179407081270973
SGD(36/49): loss=0.881665522242379
SGD(37/49): loss=1.1149858538300434
SGD(38/49): loss=0.9909861539096169
SGD(39/49): loss=0.9678675273754155
SGD(40/49): loss=1.5074923899259831
SGD(41/49): loss=1.5606460448059387
SGD(42/49): loss=3.1842323804112644
SGD(43/49): loss=3.2472071794378974
SGD(44/49): loss=3.6875915843715905
SGD(45/49): loss=3.719523723055099
SGD(46/49): loss=3.7554384569295505
SGD(47/49): loss=3.729265555130969
SGD(48/49): loss=3.5073846702885025
SGD(49/49): loss=2.881609464361875
SGD(0/49): loss=0.5
SGD(1/49): loss=0.540582156360085
SGD(2/49): loss=0.5821768079106077
SGD(3/49): loss=0.5793165570093988
SGD(4/49): loss=0.5796726515253443
SGD(5/49): loss=0.5535852950925783
SGD(6/49): loss=0.6472407085788264
SGD(7/49): loss=0.6093792367835418
SGD(8/49): loss=0.597526767915794
SGD(9/49): loss=0.5898

...

SGD(9/49): loss=163234.05193196834
SGD(10/49): loss=85338591.0316656
SGD(11/49): loss=66839951.99189153
SGD(12/49): loss=58984863.97284424
SGD(13/49): loss=2759416192.233444
SGD(14/49): loss=126360834474.45131
SGD(15/49): loss=134470640048.62386
SGD(16/49): loss=132743261640.35896
SGD(17/49): loss=130367243277.74916
SGD(18/49): loss=824164671217.6001
SGD(19/49): loss=45173280833842.47
SGD(20/49): loss=6653417249587839.0
SGD(21/49): loss=6573796751931195.0
SGD(22/49): loss=2.370057094222723e+16
SGD(23/49): loss=2.036541237975719e+16
SGD(24/49): loss=5.08798010774125e+18
SGD(25/49): loss=6.483183735722564e+19
SGD(26/49): loss=6.563882901142772e+19
SGD(27/49): loss=6.26885537783809e+19
SGD(28/49): loss=6.461073887808959e+19
SGD(29/49): loss=3.126400605999599e+20
SGD(30/49): loss=6.730426987782913e+20
SGD(31/49): loss=3.661757580256253e+22
SGD(32/49): loss=7.651574433639778e+22
SGD(33/49): loss=4.2445306945529713e+24
SGD(34/49): loss=3.807538905575508e+26
SGD(35/49): loss=4.716060189295763

SGD(41/49): loss=1.1155807500813216e+29
SGD(42/49): loss=3.9686650003364746e+30
SGD(43/49): loss=2.856492163339571e+31
SGD(44/49): loss=2.8540451721720577e+31
SGD(45/49): loss=1.3619574103777549e+32
SGD(46/49): loss=9.742576376288124e+31
SGD(47/49): loss=3.1243999323768048e+32
SGD(48/49): loss=3.2741248084385015e+32
SGD(49/49): loss=3.3816816048803344e+32
SGD(0/49): loss=0.5
SGD(1/49): loss=0.5494872449557587
SGD(2/49): loss=16.81747032353063
SGD(3/49): loss=312.5865956101487
SGD(4/49): loss=311.4481714628035
SGD(5/49): loss=75914.62328407321
SGD(6/49): loss=69252.83943465182
SGD(7/49): loss=2319288.0977306548
SGD(8/49): loss=2400245.247025127
SGD(9/49): loss=1962289.097255247
SGD(10/49): loss=1780625.9099282387
SGD(11/49): loss=66048397.61833544
SGD(12/49): loss=60641603.26505202
SGD(13/49): loss=396178122.1628184
SGD(14/49): loss=11841920660.573254
SGD(15/49): loss=10566627846.008986
SGD(16/49): loss=21353129996.52033
SGD(17/49): loss=19641197208.09256
SGD(18/49): loss=17857219736.57

SGD(21/49): loss=3.2522812466099628e+16
SGD(22/49): loss=1.9627791000224657e+19
SGD(23/49): loss=3.3599830337351365e+19
SGD(24/49): loss=3.898660457365541e+19
SGD(25/49): loss=4.3412060159425905e+19
SGD(26/49): loss=5.762255406798408e+21
SGD(27/49): loss=4.379584193772026e+22
SGD(28/49): loss=1.0192590504916338e+24
SGD(29/49): loss=7.888432704619765e+23
SGD(30/49): loss=1.678329369701526e+24
SGD(31/49): loss=1.4131018932401765e+24
SGD(32/49): loss=3.0972580712679817e+25
SGD(33/49): loss=6.901663959089899e+25
SGD(34/49): loss=9.256865479203201e+25
SGD(35/49): loss=3.8666430492152345e+26
SGD(36/49): loss=1.5426046673094742e+28
SGD(37/49): loss=1.7565711024151768e+28
SGD(38/49): loss=1.640823902219268e+28
SGD(39/49): loss=1.1056466935832138e+28
SGD(40/49): loss=8.324271640838397e+28
SGD(41/49): loss=8.288872286476615e+28
SGD(42/49): loss=7.220927776311977e+29
SGD(43/49): loss=3.1036915201077997e+30
SGD(44/49): loss=8.121113540244927e+30
SGD(45/49): loss=1.2412640859542388e+31
SGD(46/49): 

SGD(90/499): loss=0.42397128743793344
SGD(91/499): loss=0.4262040481907172
SGD(92/499): loss=0.4204887944502669
SGD(93/499): loss=0.42323552737649034
SGD(94/499): loss=0.42576589242610396
SGD(95/499): loss=0.4404515621732814
SGD(96/499): loss=0.4363644889743842
SGD(97/499): loss=0.4364797217657584
SGD(98/499): loss=0.43618367449281137
SGD(99/499): loss=0.4517956882127538
SGD(100/499): loss=0.4475329421198426
SGD(101/499): loss=0.4461508631331199
SGD(102/499): loss=0.4340330321880658
SGD(103/499): loss=0.4370146569251312
SGD(104/499): loss=0.4483325489530455
SGD(105/499): loss=0.43764158094616357
SGD(106/499): loss=0.4434629655957994
SGD(107/499): loss=0.3999193927680055
SGD(108/499): loss=0.39959377362292986
SGD(109/499): loss=0.40023884166314927
SGD(110/499): loss=0.40102935596507105
SGD(111/499): loss=0.4012196278106002
SGD(112/499): loss=0.40096060196734234
SGD(113/499): loss=0.4037412908681567
SGD(114/499): loss=0.4047508520535308
SGD(115/499): loss=0.4032169903176806
SGD(116/499):

SGD(305/499): loss=0.4709619600996119
SGD(306/499): loss=0.44765296056968085
SGD(307/499): loss=0.4436651558280582
SGD(308/499): loss=0.44273274316482664
SGD(309/499): loss=0.4639761110886297
SGD(310/499): loss=0.451819656503594
SGD(311/499): loss=0.4417429358234095
SGD(312/499): loss=0.43359794807711577
SGD(313/499): loss=0.4319754076409398
SGD(314/499): loss=0.44914874953761885
SGD(315/499): loss=0.4453401891526285
SGD(316/499): loss=0.4360427731985229
SGD(317/499): loss=0.4327388174226319
SGD(318/499): loss=0.42963924741200227
SGD(319/499): loss=0.4295996893008975
SGD(320/499): loss=0.43498711763530895
SGD(321/499): loss=0.43501836929694565
SGD(322/499): loss=0.4285310261341699
SGD(323/499): loss=0.42787848563028974
SGD(324/499): loss=0.42877028239840026
SGD(325/499): loss=0.42490285152459234
SGD(326/499): loss=0.42382907056323793
SGD(327/499): loss=0.4229194523818616
SGD(328/499): loss=0.422169393532593
SGD(329/499): loss=0.42214466091327735
SGD(330/499): loss=0.4371790887293396
SG

In [106]:
gamma_opt2 = cross_validation(set2_y, set2_x, k_fold, gammas, fonction=3)
w_sgd2, loss_sgd2 = least_squares_SGD(set2_y, set2_x, gamma_opt2, max_iters=500)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt2))
print("Stochastic gradient descent regression loss {loss}".format(loss=loss_sgd2))

SGD(0/49): loss=0.5
SGD(1/49): loss=0.5
SGD(2/49): loss=0.5
SGD(3/49): loss=0.5
SGD(4/49): loss=0.5
SGD(5/49): loss=0.5
SGD(6/49): loss=0.5
SGD(7/49): loss=0.5
SGD(8/49): loss=0.5
SGD(9/49): loss=0.5
SGD(10/49): loss=0.5
SGD(11/49): loss=0.5
SGD(12/49): loss=0.5
SGD(13/49): loss=0.5
SGD(14/49): loss=0.5
SGD(15/49): loss=0.5
SGD(16/49): loss=0.5
SGD(17/49): loss=0.5
SGD(18/49): loss=0.5
SGD(19/49): loss=0.5
SGD(20/49): loss=0.5
SGD(21/49): loss=0.5
SGD(22/49): loss=0.5
SGD(23/49): loss=0.5
SGD(24/49): loss=0.5
SGD(25/49): loss=0.5
SGD(26/49): loss=0.5
SGD(27/49): loss=0.5
SGD(28/49): loss=0.5
SGD(29/49): loss=0.5
SGD(30/49): loss=0.5
SGD(31/49): loss=0.5
SGD(32/49): loss=0.5
SGD(33/49): loss=0.5
SGD(34/49): loss=0.5
SGD(35/49): loss=0.5
SGD(36/49): loss=0.5
SGD(37/49): loss=0.5
SGD(38/49): loss=0.5
SGD(39/49): loss=0.5
SGD(40/49): loss=0.5
SGD(41/49): loss=0.5
SGD(42/49): loss=0.5
SGD(43/49): loss=0.5
SGD(44/49): loss=0.5
SGD(45/49): loss=0.5
SGD(46/49): loss=0.5
SGD(47/49): loss=0.5
SG

SGD(40/49): loss=0.46475172003595544
SGD(41/49): loss=0.46740469727183004
SGD(42/49): loss=0.4689302173280255
SGD(43/49): loss=0.47877165186618215
SGD(44/49): loss=0.4833247846620289
SGD(45/49): loss=0.4645529586813013
SGD(46/49): loss=0.4658969425119256
SGD(47/49): loss=0.4665435663842303
SGD(48/49): loss=0.4753853993760434
SGD(49/49): loss=0.4743466432918652
SGD(0/49): loss=0.5
SGD(1/49): loss=0.4986501327524063
SGD(2/49): loss=0.5018678439407567
SGD(3/49): loss=0.49898062791215775
SGD(4/49): loss=0.5079551894576099
SGD(5/49): loss=0.49755009534025185
SGD(6/49): loss=0.513871565731795
SGD(7/49): loss=0.5614625295474341
SGD(8/49): loss=0.5644072815336896
SGD(9/49): loss=0.5594841184066195
SGD(10/49): loss=0.5671729998037834
SGD(11/49): loss=0.5800514848655848
SGD(12/49): loss=0.582523832523542
SGD(13/49): loss=0.6085057087077627
SGD(14/49): loss=0.6100381560128946
SGD(15/49): loss=0.6039836249334014
SGD(16/49): loss=0.588295348542911
SGD(17/49): loss=0.6092978728048458
SGD(18/49): los

SGD(40/49): loss=0.5598089715571891
SGD(41/49): loss=0.5555752642421276
SGD(42/49): loss=0.6241116377864033
SGD(43/49): loss=0.5677099161051679
SGD(44/49): loss=0.5688479270208151
SGD(45/49): loss=0.5310527386810403
SGD(46/49): loss=0.5350382689716754
SGD(47/49): loss=0.49909168346437993
SGD(48/49): loss=0.49774415249505644
SGD(49/49): loss=0.5113369197247657
SGD(0/49): loss=0.5
SGD(1/49): loss=0.5447721710452752
SGD(2/49): loss=0.7945050012117755
SGD(3/49): loss=0.6091995020452441
SGD(4/49): loss=0.6685112492476323
SGD(5/49): loss=0.603677978336454
SGD(6/49): loss=0.6177243610868297
SGD(7/49): loss=0.6443435766920536
SGD(8/49): loss=0.6232071764856922
SGD(9/49): loss=0.6838799925075119
SGD(10/49): loss=0.7011384863760942
SGD(11/49): loss=0.5822441475563954
SGD(12/49): loss=0.6062961175247084
SGD(13/49): loss=0.6076393528237173
SGD(14/49): loss=0.5693615847286759
SGD(15/49): loss=0.5719397752086028
SGD(16/49): loss=0.5762843180590655
SGD(17/49): loss=0.5968049935354864
SGD(18/49): loss

SGD(27/49): loss=0.9114783549225421
SGD(28/49): loss=0.8899988004597964
SGD(29/49): loss=0.9533281814447145
SGD(30/49): loss=0.9637239054647825
SGD(31/49): loss=0.9735676503143401
SGD(32/49): loss=0.8797755878562025
SGD(33/49): loss=0.8510875762235509
SGD(34/49): loss=0.979408269129512
SGD(35/49): loss=0.9617838709010176
SGD(36/49): loss=0.9565027523265449
SGD(37/49): loss=0.952462700320276
SGD(38/49): loss=0.9365183046882879
SGD(39/49): loss=0.9414777757806021
SGD(40/49): loss=0.9799477411993339
SGD(41/49): loss=1.0385454055215362
SGD(42/49): loss=1.0261967232023637
SGD(43/49): loss=1.7920600785202356
SGD(44/49): loss=1.403119321871885
SGD(45/49): loss=1.4511210099721046
SGD(46/49): loss=1.1263505711516275
SGD(47/49): loss=1.1238548879897063
SGD(48/49): loss=1.1667842647500544
SGD(49/49): loss=1.469074407289619
SGD(0/49): loss=0.5
SGD(1/49): loss=0.5037941363133858
SGD(2/49): loss=0.567402551002556
SGD(3/49): loss=0.7106835532925672
SGD(4/49): loss=0.7268974865145911
SGD(5/49): loss=0

SGD(23/49): loss=1.018490834198804
SGD(24/49): loss=1.0362813049058404
SGD(25/49): loss=0.8722139938167943
SGD(26/49): loss=0.871481716531083
SGD(27/49): loss=1.0457569650138336
SGD(28/49): loss=0.8110894427514436
SGD(29/49): loss=0.9863183241835012
SGD(30/49): loss=0.7183669922523388
SGD(31/49): loss=0.8670508024404484
SGD(32/49): loss=0.6889235113972831
SGD(33/49): loss=0.750167352444189
SGD(34/49): loss=0.8546850902236506
SGD(35/49): loss=0.87439570514108
SGD(36/49): loss=0.93259561710706
SGD(37/49): loss=0.9557820359399919
SGD(38/49): loss=0.8842752204946884
SGD(39/49): loss=0.7143017026044246
SGD(40/49): loss=0.7050959829495228
SGD(41/49): loss=0.7116586839794661
SGD(42/49): loss=0.6956788794313522
SGD(43/49): loss=0.7259851693478019
SGD(44/49): loss=0.7649018500982153
SGD(45/49): loss=0.7816584026412939
SGD(46/49): loss=0.8399988877718857
SGD(47/49): loss=1.0874972450434004
SGD(48/49): loss=1.0327332724292344
SGD(49/49): loss=0.8852935296159339
SGD(0/49): loss=0.5
SGD(1/49): loss

...

SGD(0/49): loss=0.5
SGD(1/49): loss=2.733590823485383
SGD(2/49): loss=5.514548052122707
SGD(3/49): loss=12.056334620993772
SGD(4/49): loss=45.668440648944696
SGD(5/49): loss=47.64265837059866
SGD(6/49): loss=1230.6714154948145
SGD(7/49): loss=2210.1578255251397
SGD(8/49): loss=124133.12030036443
SGD(9/49): loss=170265.40362882943
SGD(10/49): loss=2193402.091324335
SGD(11/49): loss=31529868.23092944
SGD(12/49): loss=30924238.561332736
SGD(13/49): loss=72283897.06837457
SGD(14/49): loss=91270564.64655581
SGD(15/49): loss=226045458.93270132
SGD(16/49): loss=535238564.6900592
SGD(17/49): loss=1149520681.5833604
SGD(18/49): loss=1377356494.5268433
SGD(19/49): loss=15700451193.56707
SGD(20/49): loss=125123142788.03386
SGD(21/49): loss=1005163696969.3685
SGD(22/49): loss=5882130845520.227
SGD(23/49): loss=21611759391913.605
SGD(24/49): loss=199745038981014.88
SGD(25/49): loss=222772379194346.44
SGD(26/49): loss=532048529833746.4
SGD(27/49): loss=2.296038736652992e+17
SGD(28/49): loss=1.620457

SGD(26/49): loss=324930960134.056
SGD(27/49): loss=3992626996419.1484
SGD(28/49): loss=422681487946899.94
SGD(29/49): loss=336486762365653.8
SGD(30/49): loss=1.0838800687197128e+16
SGD(31/49): loss=1.951630113092342e+17
SGD(32/49): loss=3.5427204185123405e+17
SGD(33/49): loss=4.353231200059688e+19
SGD(34/49): loss=1.0890046888120025e+20
SGD(35/49): loss=1.0618434702609855e+20
SGD(36/49): loss=4.360318141736368e+20
SGD(37/49): loss=4.364102295677068e+20
SGD(38/49): loss=1.5156450808827053e+21
SGD(39/49): loss=2.3184368735072608e+22
SGD(40/49): loss=1.204950384047699e+24
SGD(41/49): loss=3.444962577666772e+26
SGD(42/49): loss=2.3298960666917437e+27
SGD(43/49): loss=1.0928921548915898e+28
SGD(44/49): loss=2.963965734365929e+29
SGD(45/49): loss=2.444296887338312e+29
SGD(46/49): loss=2.2204589605777596e+29
SGD(47/49): loss=5.25657547467473e+30
SGD(48/49): loss=6.38091226838561e+30
SGD(49/49): loss=6.057838325181091e+30
SGD(0/49): loss=0.5
SGD(1/49): loss=9.569078390795644
SGD(2/49): loss=8.

SGD(47/49): loss=1.1233057455224069e+30
SGD(48/49): loss=1.5591325748829965e+30
SGD(49/49): loss=2.1207240613901038e+32
SGD(0/49): loss=0.5
SGD(1/49): loss=4.37353345655754
SGD(2/49): loss=6.8646165575800975
SGD(3/49): loss=6.591481818127948
SGD(4/49): loss=11.862978618717879
SGD(5/49): loss=44.99954250177872
SGD(6/49): loss=44.80104305733443
SGD(7/49): loss=278.0581288628991
SGD(8/49): loss=230.0794591617584
SGD(9/49): loss=232.07367297705682
SGD(10/49): loss=309.22265972658613
SGD(11/49): loss=22513.78925757161
SGD(12/49): loss=124172.59567320846
SGD(13/49): loss=433696.0864061097
SGD(14/49): loss=102578051.01809417
SGD(15/49): loss=316153330.9067974
SGD(16/49): loss=788278218.3205689
SGD(17/49): loss=11242182906.762453
SGD(18/49): loss=10880787656.845188
SGD(19/49): loss=20948754810.73263
SGD(20/49): loss=300266984703.97437
SGD(21/49): loss=716595571530.2705
SGD(22/49): loss=3672930373864.609
SGD(23/49): loss=73143350613393.06
SGD(24/49): loss=1822543352248645.0
SGD(25/49): loss=1.7

SGD(173/499): loss=0.5454455357925173
SGD(174/499): loss=0.5411575087875354
SGD(175/499): loss=0.5180672230512732
SGD(176/499): loss=0.5197448277647254
SGD(177/499): loss=0.5163589446703036
SGD(178/499): loss=0.5268825812715864
SGD(179/499): loss=0.5281670733341886
SGD(180/499): loss=0.5171026384283403
SGD(181/499): loss=0.5243565433223162
SGD(182/499): loss=0.5261883632927374
SGD(183/499): loss=0.5094586615024118
SGD(184/499): loss=0.5227778211785903
SGD(185/499): loss=0.5104096327937907
SGD(186/499): loss=0.5137621619590624
SGD(187/499): loss=0.5175565097119013
SGD(188/499): loss=0.5032790558466351
SGD(189/499): loss=0.4857546197029648
SGD(190/499): loss=0.48382026873544476
SGD(191/499): loss=0.48864613855894445
SGD(192/499): loss=0.4937009757576556
SGD(193/499): loss=0.49102216561788353
SGD(194/499): loss=0.4937917273440146
SGD(195/499): loss=0.4979252041051199
SGD(196/499): loss=0.5054940235611913
SGD(197/499): loss=0.5086564592543267
SGD(198/499): loss=0.5097465792805828
SGD(199/4

SGD(394/499): loss=0.6480971375482216
SGD(395/499): loss=0.6519642526608955
SGD(396/499): loss=0.6601745733053457
SGD(397/499): loss=0.6906365044538016
SGD(398/499): loss=0.6854423580553765
SGD(399/499): loss=0.684953067511231
SGD(400/499): loss=0.686397803452415
SGD(401/499): loss=0.698229124449543
SGD(402/499): loss=0.5461407222677221
SGD(403/499): loss=0.5504770414035931
SGD(404/499): loss=0.5199932049383652
SGD(405/499): loss=0.4884414564640872
SGD(406/499): loss=0.4962954242568916
SGD(407/499): loss=0.5053354250361315
SGD(408/499): loss=0.5266106327138707
SGD(409/499): loss=0.5293709103470667
SGD(410/499): loss=0.5255202398265223
SGD(411/499): loss=0.523534035626043
SGD(412/499): loss=0.4989195831403297
SGD(413/499): loss=0.46677700464685407
SGD(414/499): loss=0.4662479849685557
SGD(415/499): loss=0.46260374093967455
SGD(416/499): loss=0.4609173082054153
SGD(417/499): loss=0.46419011051836556
SGD(418/499): loss=0.4705363006507145
SGD(419/499): loss=0.46675947685950864
SGD(420/499)

In [107]:
gamma_opt3 = cross_validation(set3_y, set3_x, k_fold, gammas, fonction=3)
w_sgd3, loss_sgd3 = least_squares_SGD(set3_y, set3_x, gamma_opt3, max_iters=500)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt3))
print("Stochastic gradient descent regression loss {loss}".format(loss=loss_sgd3))

SGD(0/49): loss=0.5
SGD(1/49): loss=0.5
SGD(2/49): loss=0.5
SGD(3/49): loss=0.5
SGD(4/49): loss=0.5
SGD(5/49): loss=0.5
SGD(6/49): loss=0.5
SGD(7/49): loss=0.5
SGD(8/49): loss=0.5
SGD(9/49): loss=0.5
SGD(10/49): loss=0.5
SGD(11/49): loss=0.5
SGD(12/49): loss=0.5
SGD(13/49): loss=0.5
SGD(14/49): loss=0.5
SGD(15/49): loss=0.5
SGD(16/49): loss=0.5
SGD(17/49): loss=0.5
SGD(18/49): loss=0.5
SGD(19/49): loss=0.5
SGD(20/49): loss=0.5
SGD(21/49): loss=0.5
SGD(22/49): loss=0.5
SGD(23/49): loss=0.5
SGD(24/49): loss=0.5
SGD(25/49): loss=0.5
SGD(26/49): loss=0.5
SGD(27/49): loss=0.5
SGD(28/49): loss=0.5
SGD(29/49): loss=0.5
SGD(30/49): loss=0.5
SGD(31/49): loss=0.5
SGD(32/49): loss=0.5
SGD(33/49): loss=0.5
SGD(34/49): loss=0.5
SGD(35/49): loss=0.5
SGD(36/49): loss=0.5
SGD(37/49): loss=0.5
SGD(38/49): loss=0.5
SGD(39/49): loss=0.5
SGD(40/49): loss=0.5
SGD(41/49): loss=0.5
SGD(42/49): loss=0.5
SGD(43/49): loss=0.5
SGD(44/49): loss=0.5
SGD(45/49): loss=0.5
SGD(46/49): loss=0.5
SGD(47/49): loss=0.5
SG

SGD(20/49): loss=0.5173370392306283
SGD(21/49): loss=0.5128502450438638
SGD(22/49): loss=0.5180769910279419
SGD(23/49): loss=0.5186084552011253
SGD(24/49): loss=0.5191736062007779
SGD(25/49): loss=0.518606248746922
SGD(26/49): loss=0.5265180849407397
SGD(27/49): loss=0.49873506211390745
SGD(28/49): loss=0.47164864403235046
SGD(29/49): loss=0.4734400127079255
SGD(30/49): loss=0.48042630153658866
SGD(31/49): loss=0.49560073612534716
SGD(32/49): loss=0.47879783757906075
SGD(33/49): loss=0.47701011791476405
SGD(34/49): loss=0.4696087611619476
SGD(35/49): loss=0.46530878570470174
SGD(36/49): loss=0.4508122285850296
SGD(37/49): loss=0.44837173791486223
SGD(38/49): loss=0.4499767063996196
SGD(39/49): loss=0.48608521613361716
SGD(40/49): loss=0.47451611722132797
SGD(41/49): loss=0.4719757005866369
SGD(42/49): loss=0.47373173675020075
SGD(43/49): loss=0.4713646903956799
SGD(44/49): loss=0.4857393257526666
SGD(45/49): loss=0.47005465158892346
SGD(46/49): loss=0.460330109139019
SGD(47/49): loss=0

SGD(17/49): loss=0.49685596690232525
SGD(18/49): loss=0.5273326353171187
SGD(19/49): loss=0.4822397431219299
SGD(20/49): loss=0.48944265034154155
SGD(21/49): loss=0.5087639824011807
SGD(22/49): loss=0.4898299043959642
SGD(23/49): loss=0.4368789871088764
SGD(24/49): loss=0.46712213101031497
SGD(25/49): loss=0.4544765262405692
SGD(26/49): loss=0.45560424639126285
SGD(27/49): loss=0.44542797877871526
SGD(28/49): loss=0.46124868296474264
SGD(29/49): loss=0.462191581033427
SGD(30/49): loss=0.5074170186088439
SGD(31/49): loss=0.5101909778155975
SGD(32/49): loss=0.5413370115967108
SGD(33/49): loss=0.537409824141077
SGD(34/49): loss=0.5344045790298537
SGD(35/49): loss=0.533499694738458
SGD(36/49): loss=0.5265691548053725
SGD(37/49): loss=0.530793061760673
SGD(38/49): loss=0.5287851917257826
SGD(39/49): loss=0.5308871281427701
SGD(40/49): loss=0.47971934119825654
SGD(41/49): loss=0.47151251063972544
SGD(42/49): loss=0.46725158178036846
SGD(43/49): loss=0.4679601334913177
SGD(44/49): loss=0.4494

SGD(6/49): loss=0.8447543214412147
SGD(7/49): loss=0.5189661068642981
SGD(8/49): loss=0.5401111031569423
SGD(9/49): loss=0.5198207545954916
SGD(10/49): loss=0.5170407773611251
SGD(11/49): loss=0.5331077740188295
SGD(12/49): loss=0.8674027733182293
SGD(13/49): loss=0.8685691347961052
SGD(14/49): loss=0.5560281312216853
SGD(15/49): loss=0.6332003587453343
SGD(16/49): loss=0.9881768707532215
SGD(17/49): loss=0.9595915442692367
SGD(18/49): loss=0.9760149130510899
SGD(19/49): loss=0.9531851236589342
SGD(20/49): loss=1.0181454872956766
SGD(21/49): loss=0.8169256334097768
SGD(22/49): loss=0.7558772020367194
SGD(23/49): loss=0.7154906260943721
SGD(24/49): loss=0.646566261826904
SGD(25/49): loss=0.6469399449797352
SGD(26/49): loss=0.6785437333165526
SGD(27/49): loss=0.6775909194218346
SGD(28/49): loss=0.7480684655329074
SGD(29/49): loss=0.7627447931514657
SGD(30/49): loss=0.7395017808555069
SGD(31/49): loss=0.844771618948
SGD(32/49): loss=1.0929241960232525
SGD(33/49): loss=1.25661261239416
SGD

SGD(45/49): loss=3.2357045081634044
SGD(46/49): loss=2.7045487163251285
SGD(47/49): loss=2.7706011964101056
SGD(48/49): loss=2.4151190941949676
SGD(49/49): loss=2.4450045418291344
SGD(0/49): loss=0.5
SGD(1/49): loss=0.5015466416675145
SGD(2/49): loss=0.5980538625000413
SGD(3/49): loss=0.5269207730180006
SGD(4/49): loss=0.5370417574714392
SGD(5/49): loss=0.7047143865722405
SGD(6/49): loss=0.954758669687056
SGD(7/49): loss=0.9089805713707925
SGD(8/49): loss=1.1930396937220764
SGD(9/49): loss=1.3150209694649668
SGD(10/49): loss=1.1893482819774697
SGD(11/49): loss=1.2035592336569705
SGD(12/49): loss=1.25972774673904
SGD(13/49): loss=1.1231993250364445
SGD(14/49): loss=1.1822717822280115
SGD(15/49): loss=2.0662107653705655
SGD(16/49): loss=1.4450993456367087
SGD(17/49): loss=13.89448651957023
SGD(18/49): loss=14.119076379082712
SGD(19/49): loss=14.109949479581879
SGD(20/49): loss=13.998931592869372
SGD(21/49): loss=11.465874448376299
SGD(22/49): loss=7.755003866524828
SGD(23/49): loss=7.350

...

SGD(37/49): loss=2.4416864306902447e+26
SGD(38/49): loss=2.446841993332303e+28
SGD(39/49): loss=2.3155143578457396e+28
SGD(40/49): loss=1.7791509505053881e+28
SGD(41/49): loss=2.2345802895424945e+30
SGD(42/49): loss=3.7016253674329107e+31
SGD(43/49): loss=3.5539847458121024e+31
SGD(44/49): loss=8.136859637153397e+31
SGD(45/49): loss=8.149060756020624e+31
SGD(46/49): loss=7.7597547125206e+32
SGD(47/49): loss=9.989251424884994e+32
SGD(48/49): loss=9.816028107882734e+32
SGD(49/49): loss=9.518754048504093e+32
SGD(0/49): loss=0.5
SGD(1/49): loss=2.8771127854972036
SGD(2/49): loss=1188.9097964987861
SGD(3/49): loss=1391.8142207098017
SGD(4/49): loss=4673.396234826354
SGD(5/49): loss=4688.655729226832
SGD(6/49): loss=68441.77132081233
SGD(7/49): loss=329296.0073087465
SGD(8/49): loss=3324787.3023055363
SGD(9/49): loss=3056518.07344264
SGD(10/49): loss=3189618.642369228
SGD(11/49): loss=13594927.92408649
SGD(12/49): loss=172877445.40914044
SGD(13/49): loss=2476083643.043424
SGD(14/49): loss=23

SGD(10/49): loss=382383773287.56055
SGD(11/49): loss=3374144222066.784
SGD(12/49): loss=3278113019418.945
SGD(13/49): loss=31782865161391.45
SGD(14/49): loss=42249986060848.66
SGD(15/49): loss=207668335825662.7
SGD(16/49): loss=187932139818414.75
SGD(17/49): loss=1.1251760577804368e+16
SGD(18/49): loss=1.1123859305301488e+16
SGD(19/49): loss=1.051277417269819e+16
SGD(20/49): loss=1.634298320600154e+19
SGD(21/49): loss=3.683250562958518e+19
SGD(22/49): loss=1.6999293323541105e+21
SGD(23/49): loss=6.226133534247544e+21
SGD(24/49): loss=1.1501595155195434e+22
SGD(25/49): loss=9.33110875559799e+22
SGD(26/49): loss=3.1375894429354384e+24
SGD(27/49): loss=8.790937465838696e+24
SGD(28/49): loss=3.3560975676761904e+25
SGD(29/49): loss=3.612680547025423e+25
SGD(30/49): loss=2.0778904574793738e+27
SGD(31/49): loss=9.215626248837916e+27
SGD(32/49): loss=2.3278313214739364e+29
SGD(33/49): loss=2.104621602033465e+29
SGD(34/49): loss=1.1403443351235428e+30
SGD(35/49): loss=6.174127099615262e+30
SGD(

SGD(41/49): loss=1.4900849099037723e+30
SGD(42/49): loss=3.0305335386415713e+31
SGD(43/49): loss=4.673503448408534e+31
SGD(44/49): loss=2.078638487083645e+32
SGD(45/49): loss=2.3532432435228536e+33
SGD(46/49): loss=3.846195375805341e+34
SGD(47/49): loss=1.4936908352600613e+35
SGD(48/49): loss=1.737567432464361e+35
SGD(49/49): loss=1.709521371945004e+35
SGD(0/49): loss=0.5
SGD(1/49): loss=0.3935696950032373
SGD(2/49): loss=0.4360584660649577
SGD(3/49): loss=0.6728403684629362
SGD(4/49): loss=65.2547903581348
SGD(5/49): loss=60.93086951323646
SGD(6/49): loss=161.30477399837653
SGD(7/49): loss=812.0386086242684
SGD(8/49): loss=3838.349465781441
SGD(9/49): loss=11725.901303429264
SGD(10/49): loss=116518.10968196727
SGD(11/49): loss=179089.24723063546
SGD(12/49): loss=204049.6844286438
SGD(13/49): loss=202657.58276908356
SGD(14/49): loss=854944.774224834
SGD(15/49): loss=840314.7179978341
SGD(16/49): loss=875849.582964817
SGD(17/49): loss=893978.2132754095
SGD(18/49): loss=1222927.728576158

SGD(70/499): loss=0.40738370801544266
SGD(71/499): loss=0.40679379223313855
SGD(72/499): loss=0.40619721762285427
SGD(73/499): loss=0.41182526365921757
SGD(74/499): loss=0.4133421228390371
SGD(75/499): loss=0.4221700782877501
SGD(76/499): loss=0.42201696391067667
SGD(77/499): loss=0.42174634045581927
SGD(78/499): loss=0.42194805564780613
SGD(79/499): loss=0.42419944586968594
SGD(80/499): loss=0.42639934678154995
SGD(81/499): loss=0.4101935112741057
SGD(82/499): loss=0.4109427095011222
SGD(83/499): loss=0.4127452150270201
SGD(84/499): loss=0.41382678757719965
SGD(85/499): loss=0.42033363650538025
SGD(86/499): loss=0.4227428545547332
SGD(87/499): loss=0.4230213465407164
SGD(88/499): loss=0.4230063322486475
SGD(89/499): loss=0.4273828949194637
SGD(90/499): loss=0.4378667115484275
SGD(91/499): loss=0.4444298527007772
SGD(92/499): loss=0.446848087842561
SGD(93/499): loss=0.44347749785958024
SGD(94/499): loss=0.4456368318064483
SGD(95/499): loss=0.43939606088010863
SGD(96/499): loss=0.451601

SGD(292/499): loss=0.3942599760577772
SGD(293/499): loss=0.39366902443792234
SGD(294/499): loss=0.3948095577697967
SGD(295/499): loss=0.4014583118335187
SGD(296/499): loss=0.4014711736306258
SGD(297/499): loss=0.4059655273640763
SGD(298/499): loss=0.4073885930941292
SGD(299/499): loss=0.44081478834222176
SGD(300/499): loss=0.463088497314299
SGD(301/499): loss=0.45911500296240265
SGD(302/499): loss=0.4576912562411714
SGD(303/499): loss=0.4491850072048232
SGD(304/499): loss=0.4434425032194506
SGD(305/499): loss=0.41908633317531935
SGD(306/499): loss=0.4219134395975068
SGD(307/499): loss=0.4191931251120538
SGD(308/499): loss=0.4128926180826126
SGD(309/499): loss=0.4091792621215947
SGD(310/499): loss=0.4109709891033469
SGD(311/499): loss=0.4071576103907759
SGD(312/499): loss=0.4074170482797639
SGD(313/499): loss=0.40038715295588684
SGD(314/499): loss=0.4025332754053051
SGD(315/499): loss=0.40443798488210464
SGD(316/499): loss=0.4055113612513805
SGD(317/499): loss=0.4092437477928902
SGD(318

### Logistic regression

In [112]:
k_fold = 4
max_iters = 200
gammas = np.arange(0, 0.5, 0.01)
tX_log = log_distribution(tX, to_log)
set1_x, set1_y, set1_ids, set2_x, set2_y, set2_ids, set3_x, set3_y, set3_ids = separate_sets(tX_log, y, ids)

set1_x = outliers(set1_x, -999)
set1_x = filtering_with_mean_bis(set1_x, set1_y)
#set1_x = filtering_with_mean(set1_x)
#set1_x = normalize(set1_x_gd)
set1_x = std(set1_x)

set2_x = outliers(set2_x, -999)
set2_x = filtering_with_mean_bis(set2_x, set2_y)
#set2_x = filtering_with_mean(set2_x)
#set2_x= normalize(set2_x)
set2_x = std(set2_x)

set3_x = outliers(set3_x, -999)
set3_x = filtering_with_mean_bis(set3_x, set3_y)
#set3_x = filtering_with_mean(set3_x)
#set2_x = normalize(set2_x)
set3_x = std(set3_x)

print('')
print("Preprocessing for logistic regression done!")

outliers ratio for each feature [0.2614574679971575, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
outliers ratio for each feature [0.09751882802022077, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
outliers ratio for each feature [0.06105344416415092, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Preprocessing for logistic regression done!


In [ ]:
initial_w = np.zeros(tX.shape[1])
gamma_opt = cross_validation(y, tX, k_fold, gammas, fonction=4)
w_lr, loss_lr = logistic_regression(y, tX, initial_w, max_iters, gamma_opt)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt))
print("logistic regression loss {loss}".format(loss=loss_lr))

In [ ]:
initial_w = np.zeros(set2_x.shape[1])
gamma_opt = cross_validation(set2_y, set2_x, k_fold, gammas, fonction=4)
w_lr, loss_lr = logistic_regression(y,tx, initial_w, max_iters, gamma_opt)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt))
print("logistic regression loss {loss}".format(loss=loss_lr))

In [ ]:
initial_w = np.zeros(set3_x.shape[1])
gamma_opt = cross_validation(set3_y, set3_x, k_fold, gammas, fonction=4)
w_lr, loss_lr = logistic_regression(y,tx, initial_w, max_iters, gamma_opt)
print("Cross validation finished: optimal gamma {g}".format(g=gamma_opt))
print("logistic regression loss {loss}".format(loss=loss_lr))

## Generate predictions and save ouput in csv format for submission:

In [109]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

#### Least squares submissions

In [ ]:
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
OUTPUT_PATH = '../data/pred_ls.csv' 
tX_test_ls = cut(std(filtering_with_mean(tX_test)),[15,18,20])
tX_test_ls = build_poly(tX_test_ls, degree_ls)
y_pred = predict_labels(w_ls, tX_test_ls)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH

In [ ]:
OUTPUT_PATH = '../data/pred_set_ls.csv' 
DATA_TEST_PATH = '../data/test.csv'
y_test , tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

tX_test = cut(tX_test, [15,18,20])

set1_x, _, set1_ids, set2_x, _, set2_ids, set3_x, _, set3_ids = separate_sets(tX_test, y_test, ids_test, 22-3)

def filtering_test_ls (set_x, degree_rr):
    print(set_x.shape)
    set_x = outliers(set_x, -999)
    print(set_x.shape)
    set_x = std(filtering_with_mean(set_x))
    set_x = build_poly(set_x, degree_rr)
    return set_x


set1_x = filtering_test_ls(set1_x, degree_set1_ls)
set2_x = filtering_test_ls(set2_x, degree_set2_ls)
set3_x = filtering_test_ls(set3_x, degree_set3_ls)

y_pred1 = predict_labels(w_set1_ls, set1_x)
y_pred2 = predict_labels(w_set2_ls, set2_x)
y_pred3 = predict_labels(w_set3_ls, set3_x)

y_pred_ls, ids_ls = concatenate_sets(y_pred1, set1_ids, y_pred2, set2_ids, y_pred3, set3_ids)
create_csv_submission(ids_ls, y_pred_ls, OUTPUT_PATH)

#### Ridge regression submissions

In [ ]:
OUTPUT_PATH = '../data/pred_set_rr.csv' 
DATA_TEST_PATH = '../data/test.csv'
y_test , tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

set1_x, _, set1_ids, set2_x, _, set2_ids, set3_x, _, set3_ids = separate_sets(tX_test, y_test, ids_test)

def filtering_test_rr (set_x, degree_rr):
    set_x = outliers(set_x, -999)
    set_x = build_poly(set_x, degree_rr)
    return set_x

set1_x = filtering_test_rr(set1_x, degree_rr_set1)
set2_x = filtering_test_rr(set2_x, degree_rr_set2)
set3_x = filtering_test_rr(set3_x, degree_rr_set3)
y_pred1 = predict_labels(w_rr_set1, set1_x)
y_pred2 = predict_labels(w_rr_set2, set2_x)
y_pred3 = predict_labels(w_rr_set3, set3_x)

y_pred_rr, ids_rr = concatenate_sets(y_pred1, set1_ids, y_pred2, set2_ids, y_pred3, set3_ids)
create_csv_submission(ids_rr, y_pred_rr, OUTPUT_PATH)

#### Gradient descent submissions

In [95]:
tX_test_log = log_distribution(tX_test, to_log)
test_set1_x, _, test_set1_ids, test_set2_x, _, test_set2_ids, test_set3_x, _, test_set3_ids = separate_sets(tX_test_log, _, ids_test)

test_set1_x = outliers(test_set1_x, -999)
test_set1_x = filtering_with_mean(test_set1_x)
test_set1_x = std(test_set1_x)

test_set2_x = outliers(test_set2_x, -999)
test_set2_x = filtering_with_mean(test_set2_x)
test_set2_x = std(test_set2_x)

test_set3_x = outliers(test_set3_x, -999)
test_set3_x = filtering_with_mean(test_set3_x)
test_set3_x = std(test_set3_x)

outliers ratio for each feature [0.26054480387588036, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
outliers ratio for each feature [0.09834148900979822, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
outliers ratio for each feature [0.060335344108509326, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [98]:
OUTPUT_PATH = '../data/pred_gd.csv' # TODO: fill in desired name of output file for submission
y_pred1 = predict_labels(w_gd1, test_set1_x)
y_pred2 = predict_labels(w_gd2, test_set2_x)
y_pred3 = predict_labels(w_gd3, test_set3_x)
y_pred = np.concatenate((y_pred1, y_pred2, y_pred3), axis=0)
ids_test = np.concatenate((test_set1_ids, test_set2_ids, test_set3_ids), axis=0)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

#### Stochastic gradient descent submissions

In [110]:
tX_test_log = log_distribution(tX_test, to_log)
test_set1_x, _, test_set1_ids, test_set2_x, _, test_set2_ids, test_set3_x, _, test_set3_ids = separate_sets(tX_test_log, _, ids_test)

test_set1_x = outliers(test_set1_x, -999)
test_set1_x = filtering_with_mean(test_set1_x)
test_set1_x = std(test_set1_x)

test_set2_x = outliers(test_set2_x, -999)
test_set2_x = filtering_with_mean(test_set2_x)
test_set2_x = std(test_set2_x)

test_set3_x = outliers(test_set3_x, -999)
test_set3_x = filtering_with_mean(test_set3_x)
test_set3_x = std(test_set3_x)

outliers ratio for each feature [0.26054480387588036, 3.9567744374785676e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
outliers ratio for each feature [0.09834148900979822, 4.562616204131449e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
outliers ratio for each feature [0.060335344108509326, 5.439972921023682e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [111]:
OUTPUT_PATH = '../data/pred_gd.csv' # TODO: fill in desired name of output file for submission
y_pred1 = predict_labels(w_gd1, test_set1_x)
y_pred2 = predict_labels(w_gd2, test_set2_x)
y_pred3 = predict_labels(w_gd3, test_set3_x)
y_pred = np.concatenate((y_pred1, y_pred2, y_pred3), axis=0)
ids_test = np.concatenate((test_set1_ids, test_set2_ids, test_set3_ids), axis=0)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)